In [1]:
model = "llama3.2:1b"

#### Task 1: Simple Chain with Retrieval

**Objective:**

Implement a simple RAG chain with ChatOllama, HuggingFaceEmbeddings and Chroma. 

Process: 

1. Retrieve documents from chroma db based on query
2. Invoke chain with retrieved documents as input

**Task Description:**

- load llm model via ollama
- load embedding model via ollama with `ollama pull pull bge-m3` (if not yet done)
- create chroma db client
- create prompt template for summarization
- create simple chain with following steps: retrieved documents, prompt, model, output parser
- create query and perform similarity search with a query
- invoke chain and pass retrieved documents to the chain


**Useful links:**

- [RAG with Ollama](https://python.langchain.com/v0.2/docs/tutorials/local_rag/)
- [Streaming in Langchain](https://python.langchain.com/docs/concepts/streaming/)


In [2]:
from langchain_ollama import ChatOllama

# ADD HERE YOUR CODE
model = ChatOllama(model="llama3.2:1b")

In [3]:
from langchain_ollama import OllamaEmbeddings

# ADD HERE YOUR CODE
embedding_model = OllamaEmbeddings(model="bge-m3")

In [1]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# Create a collection
# ADD HERE YOUR CODE
collection_name = "ai_model_book"
collection = client.get_collection(name=collection_name)

# Create chromadb
# ADD HERE YOUR CODE
vector_db_from_client = Chroma(client=client,
        collection_name=collection_name,
        embedding_function=embedding_model
        )

ValueError: {"detail":"Not Found"} (trace ID: 0)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = prompt | model | StrOutputParser()

In [ ]:
search_query = "Types of Machine Learning Systems"

# ADD HERE YOUR CODE
# Perform vector search
docs = vector_db_from_client.similarity_search(search_query, k=3)


print(docs)

In [ ]:
chain.invoke({"docs": formatted_docs})

In [ ]:
# Simple stream the chain output
for chunk in chain.stream({"docs": formatted_docs}):
    print(chunk, end="", flush=True)

In [ ]:
# More complex async event streaming
async for event in chain.astream_events({"docs": formatted_docs}, version="v2"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)

#### Task 2: Q&A with RAG

**Objective:**

Implement a Q/A retrieval chain with ChatOllama, HuggingFaceEmbeddings and Chroma

**Task Description:**

- create RAG-Q/A prompt template
- create retriever from vector db client (instead of manually passing in docs, we automatically retrieve them from our vector store based on the user question)
- create simple chain with following steps: retriever, formatting retrieved docs, user question, prompt, model, output parser
- create question for Q/A retrieval chain
- invoke chain and with question

**Useful links:**

- [RAG with Ollama](https://python.langchain.com/v0.2/docs/tutorials/local_rag/)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

prompt_template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

# ADD HERE YOUR CODE
rag_prompt = ChatPromptTemplate.from_template(prompt_template)

# ADD HERE YOUR CODE
retriever = vector_db_from_client.as_retriever(search_kwargs={"k": 3})

# ADD HERE YOUR CODE
qa_rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | rag_prompt
        | model
        | StrOutputParser()
    )

In [ ]:
qa_rag_chain

In [ ]:
question = "What is supervised learning?"

# ADD HERE YOUR CODE
qa_rag_chain.invoke(question)

In [ ]:
# More complex async event streaming
async for event in qa_rag_chain.astream_events(question, version="v2"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)

#### Alternative: Using pre-built ConversationalRetrievalChain Class

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [ ]:
retriever = vector_db_from_client.as_retriever()
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
qa_chain = ConversationalRetrievalChain.from_llm(
    model, retriever=retriever, memory=memory, verbose=False
)

In [ ]:
# More complex async event streaming
async for event in qa_chain.astream_events("What is supervised learning?", version="v2"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)

In [ ]:
# More complex async event streaming
async for event in qa_chain.astream_events("Which algorithms can be used there?", version="v2"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)